In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Prethesis-model/kaggle.json ~/.kaggle/
! chmod 600 /content/drive/MyDrive/Prethesis-model/kaggle.json
! kaggle datasets list
! kaggle competitions download plant-pathology-2021-fgvc8
! unzip plant-pathology-2021-fgvc8.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: train_images/e00c97b2de4749ce.jpg  
  inflating: train_images/e00df890cfe62736.jpg  
  inflating: train_images/e00ed0bf43a87f25.jpg  
  inflating: train_images/e00f759d65d52887.jpg  
  inflating: train_images/e00f934d9e50ad5b.jpg  
  inflating: train_images/e00fce341cb63da5.jpg  
  inflating: train_images/e010c7c35ecf3d1c.jpg  
  inflating: train_images/e01112939f783fec.jpg  
  inflating: train_images/e01592da4d71b8be.jpg  
  inflating: train_images/e0189acaf8cf80fd.jpg  
  inflating: train_images/e0189e4999bdd0fa.jpg  
  inflating: train_images/e01ae16d4a759cb5.jpg  
  inflating: train_images/e01dc39ccb2b4ccb.jpg  
  inflating: train_images/e01e42d90c66f3f3.jpg  
  inflating: train_images/e01e870bec65f40f.jpg  
  inflating: train_images/e01f17e0187fb80f.jpg  
  inflating: train_images/e01f503cb752e6c5.jpg  
  inflating: train_images/e024b69a784e9fc3.jpg  
  inflating: train_images/e026b4c23c9fcfc4.jpg  
  inflating:

# **Data**

In [2]:
import pandas as pd
train=pd.read_csv('/content/train.csv')
train['labels'] = train['labels'].apply(lambda string: string.split(' '))


train['labels']
from sklearn.preprocessing import MultiLabelBinarizer

s = list(train['labels'])
mlb = MultiLabelBinarizer()
trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=train.index)
labels = pd.concat([train['image'], trainx], axis=1)
labels.to_csv('multilabel.csv')
import pandas as pd
a=pd.read_csv('/content/multilabel.csv').drop(['Unnamed: 0'],axis=1)
index=a.sum(axis=1)
a.drop(index,inplace=True) # Drop multi label rows
#If choose the first label that an image in to present its main label
def defineLable(row):
    count=0
    for cols in row.index[1:]:
        if row[cols]!=0:
            return count
        else:
            count+=1
a['labels']=a.apply(defineLable,axis=1)
a['labels'].value_counts() #-> 0
b=a.groupby(['labels'])
b=b.head(1000)
b['labels'].value_counts() #-> 0
b.to_csv('data1k.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  app.launch_new_instance()


# **MODEL**

In [3]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torchvision.models as models
import torch
import copy
import torch.nn.functional as F
#from model import net
import random
import multiprocessing as mp
import os
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision import datasets,transforms
import random
from torch import optim,ceil
import torchvision.models as models
import torch.nn.functional as F
from torch import nn
import pickle
import numpy as np
import torchvision.models as models
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
class Client():
    def __init__(self,id=9999,dataClass=0,initModel=models.mobilenet_v2(pretrained=True),device=None,dataset=None,parameters=None):
        self.id=id
        self.dataClass=dataClass
        self.curModel=initModel
        #For test
        #self.curModel.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=15, bias=True))
        self.device=device
        #Parameters:
        self.dataset=plantdisease(isTest=False,data=dataset)
        #self.dataset=dataGeneMNIST.MNIST(isTest=False,data=dataset)
        self.learning_rate=parameters['learning_rate']
        self.momentum=parameters['momentum']
        self.batch_size=parameters['batch_size']
        self.local_epoch=parameters['local_epoch']
        self.weight_decay=parameters['weight_decay']
        self.lossHist=[]
    def __len__(self):
        return len(self.dataset)
    def set_learningrate(self,learning_rate):
        self.learning_rate=learning_rate
    def clientID(self):
        print("ID of client: {}".format(self.id))
    def processID(self):
        print("ID of process: {}".format(os.getpid()))



    def client_train(self,lr=0.0003,epoch=10,optimizer=None):
        print("Client , data: {} trainning".format(self.id))
        
        model = self.curModel
        self.optimizer =optim.SGD(model.classifier.parameters(),lr=self.learning_rate,momentum=self.momentum)
        ####

        dataLoader=torch.utils.data.DataLoader(self.dataset,
                                         batch_size=int(self.batch_size),
                                         shuffle=True)

        ######
        for epoc in range(epoch):
            epoc_loss = 0.0
            for data, target in dataLoader:
            #
                #data = data.repeat(1, 3, 1, 1)
            #
                self.optimizer.zero_grad()
                output = model(data)
                loss = torch.nn.CrossEntropyLoss()(output, target)
                epoc_loss += loss.item()
                loss.backward()
                self.optimizer.step()
            
            print('Client ID ', self.id, ', epoch ',
              epoc, ': ', epoc_loss / self.batch_size)
        torch.cuda.empty_cache()
    def client_trainGPU(self,epoch=0):
        lr=self.learning_rate
        
        print("Client {}, data: {} trainning".format(self.id,self.id))
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.curModel.to(device)
        #self.dataset.changeisTest(False)
        ####
        self.optimizer =optim.Adam(self.curModel.parameters(),lr=self.learning_rate,weight_decay=self.weight_decay)
        dataLoader=torch.utils.data.DataLoader(self.dataset,
                                         batch_size=int(self.batch_size),
                                         shuffle=True)

        ######
        for epoc in range(epoch):
            epoc_loss = 0.0
            for data, target in dataLoader:
            #
            #data = data.repeat(1, 3, 1, 1)
            #   
                
                data,target=data.to(device),target.to(device)
                
                
                output = self.curModel(data)
                loss = torch.nn.CrossEntropyLoss()(output, target)

                epoc_loss += loss.item()
                self.optimizer.zero_grad(set_to_none=True)
                loss.backward()
                self.optimizer.step()
            
            print('Client ID ', self.id, ', epoch ',
              epoc, ': ', epoc_loss / self.batch_size)
            self.lossHist.append(epoc_loss)
        self.curModel.to('cpu')
        
    #Hỏng
    def client_evaluate(self):
        self.curModel.eval()
        testDataset=self.dataset
        testDataset.changeisTest(True)
        testDataLoader=torch.utils.data.DataLoader(testDataset,
                                         batch_size=int(20),
                                         shuffle=True)
        test_loss,correct=0,0
        #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.curModel.to(0)
        with torch.no_grad():
            for data,labels in testDataLoader:
                data,labels=data.to(0),labels.to(0)
                outputs=self.curModel(data)
                test_loss+=torch.nn.CrossEntropyLoss()(outputs, labels).item()
                predicted=outputs.argmax(dim=1,keepdim=True)
                print(labels)
                print('-----')
                correct += predicted.eq(labels.view_as(predicted)).sum().item()
                print(predicted)
        test_loss = test_loss / len(testDataLoader)
        test_accuracy=correct/len(testDataset)
        message = f"\t[Client {(self.id)}] ...finished evaluation!\
            \n\t=> Test loss: {test_loss:.4f}\
            \n\t=> Test accuracy: {100. * test_accuracy:.2f}%\n"
        print(message, flush=True)
#a=Client(dataClass=1)
#a.client_trainGPU(epoch=30)
#def saveServer(server):
#    import pickle
#    with open('server.pickle', 'wb') as f:
#        pickle.dump(server, f)
#saveServer(a)
#a.client_evaluate()

class server():
    def __init__(self,tensorboard,parameters):
        self.list_Client=[]
        #self.centralModel=MultilabelClassifier()
        self.centralModel=models.mobilenet_v2(pretrained=True)
        #self.centralModel=CNN(name='CNN',in_channels=1,hidden_channels=32,num_hiddens=512,num_classes=10)
        
        #torch.nn.init.xavier_uniform(self.centralModel.fc1.weight)
        #self.centralModel.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=15, bias=True))
        self.centralModel.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=6, bias=True))
        #torch.nn.init.xavier_uniform(self.centralModel.classifier[1].weight)
        self.centralDataset=plantdisease(isTest=False,rank=None,central=True,num_client=parameters['server']['num_clients'])
        #self.writer=tensorboard
        #self.centralDataset=dataGeneMNIST.MNIST(isTest=False,rank=None,central=True)
        self.parameters=parameters
        self.num_clients=parameters['server']['num_clients']
        self.num_round=parameters['server']['num_round']
        self.num_client_perRound=parameters['server']['num_client_perRound']
        self.isParallel=parameters['server']['isParallel']
        self.local_epoch=parameters['server']['local_epoch']
        self.accuracyHist=[]
        self.lossHist=[]
    def __len__(self):
        return len(self.list_Client)

    def changeLR(self,lr):
        for client in self.list_Client:
            client.set_learningrate(lr)
    def generateClient(self):
        #print('List class {} '.format(dices_classs))
        for i in range(0,self.num_clients):

            #makeClientDataSet(rank) -- with rank is the id of shards data
            datasetTemp=self.centralDataset.makeClientDataSet(rank=i)
            #Assign each client a shard of dataset 
            self.list_Client.append(Client(id=i,dataset=datasetTemp,initModel=copy.deepcopy(self.centralModel),parameters=self.parameters['client']))
        #for i in dices_classs:
        #    if(i>14):
        #        datasetTemp=self.centralDataset.makeClientDataSet(rank=random.randint(0, 10))
        #    else:
        #        datasetTemp=self.centralDataset.makeClientDataSet(rank=i)
        #    self.list_Client.append(client.Client(id=i,dataset=datasetTemp,initModel=copy.deepcopy(self.centralModel),parameters=self.parameters['client']))

    def updateClientModel(self):
        params=self.collectModel()
        paraAverage=self.averageModel(params)
        self.centralModel.load_state_dict(paraAverage)
        
        for idx in self.list_Client:
            #Sửa thử xem load_state_dict lcu1 trước làm việc có đúng ko
            #client.curModel.load_state_dict(paraAverage)
            idx.curModel=copy.deepcopy(self.centralModel)
        print('All clients updated')
    def collectModel(self):
        model_params=[]
        for client in self.Dices:
            model_params.append(client.curModel.state_dict())
        return model_params
    def averageModel(self,params):
        total_len=0
        for i in self.Dices:
            total_len+=len(i)
        listClient_thisRound=self.Dices
        
        #coef=torch.tensor([len(listClient_thisRound[i])/total_len for i in range(0,len(listClient_thisRound)-1)])
        #coef=1/len(listClient_thisRound)
        global_model_weights = copy.deepcopy(self.centralModel.state_dict())
        for key in global_model_weights:
            global_model_weights[key] = torch.zeros_like(
                global_model_weights[key])
        
        #server_params=copy.deepcopy(self.centralModel.parameters())
        for user in listClient_thisRound:
            local_model_weights = copy.deepcopy(user.curModel.state_dict())
            for key in global_model_weights:
                if str(global_model_weights[key].type())!='torch.LongTensor':
                    global_model_weights[key] += (1 / len(listClient_thisRound)) * local_model_weights[key]
        return global_model_weights
        print('finish average')
        #coef=1
        # from collections import OrderedDict
        # paraOut=OrderedDict()
        # for key in params[0]:
        #     if key=='classifier.1.weight' or key=='classifier.1.bias'  :
        #         paraOut[key] = sum([params[i][key]*coef[i] for i in range(0,len(params)-1)])
        #     else:
        #         paraOut[key]=params[0][key]
            #paraOut[key] = sum([params[i][key]*coef[i] for i in range(0,len(params)-1)])
        #return paraOut
        #for i in range(0,len(params)-1):
        #    for key in params[0]:
        #        if i==0:
         #           paraOut[key]=params[i][key]*coef[i]
         #       else:
         #           paraOut[key]+=params[i][key]*coef[i]
    def evaluate_CentralModel(self):
        model=self.centralModel
        model.eval()
        self.centralDataset.changeisTest(True)
        testDataLoader=torch.utils.data.DataLoader(self.centralDataset,
                                         batch_size=int(20),
                                         shuffle=True)
        test_loss,correct=0,0

        with torch.no_grad():
            for data,labels in testDataLoader:
                outputs=self.centralModel(data)


                test_loss+=torch.nn.CrossEntropyLoss()(outputs, labels).item()
                predicted=outputs.argmax(dim=1,keepdim=True)
                print('Labels')
                print(predicted)
                print('------------------------')
                correct += predicted.eq(labels.view_as(predicted)).sum().item()
        test_loss = test_loss / len(testDataLoader)
        test_accuracy=correct/len(self.centralDataset)
        self.accuracyHist.append(test_accuracy)

        self.lossHist.append(test_loss)
        print(self.lossHist)
        print(self.accuracyHist)
        #self.writer.add_scalar(tag='test_loss',scalar_value=test_loss)
        #self.writer.add_scalar(tag='accuracy',scalar_value=100. * test_accuracy)
        message = f"\t[Central Model ] ...finished evaluation!\
            \n\t=> Test loss: {test_loss:.4f}\
            \n\t=> Test accuracy: {100. * test_accuracy:.2f}%\n"
        print(message, flush=True)
        self.centralDataset.changeisTest(False)
        #self.writer.flush()
    # def trainPerClient(self,clientID,GPU=False):

    #     if GPU==True:
    #         self.list_Client[clientID].client_trainGPU(epoch=self.local_epoch)
    #     elif self.isParallel!='True':
    #         self.list_Client[clientID].client_train(epoch=self.local_epoch)
        #self.list_Client[clientID].client_evaluate()
    # def evaluatePerClient(self,clientID):
    #     self.list_Client[clientID].client_evaluate()
    # def trainClients(self):
    #     with mp.Pool(processes=len(self.list_Client)) as pool:
            
    #         pool.map(self.trainPerClient,[i for i in range(0,len(self.list_Client))])
    #         pool.map(self.evaluatePerClient,[i for i in range(0,len(self.list_Client))])
    #         self.updateClientModel()
    #         self.evaluate_CentralModel()
        print('next round')
    def trainClientsQueue(self):
        print("Start to train {} client".format(self.list_Client))
        from random import sample
        #self.Dices=[random.randint(0, len(self.list_Client)-1) for i in range(0,self.num_client_perRound)] #Random dices for client per round
        #Sample num_client_perRound from list_Client
        self.Dices=sample(self.list_Client,self.num_client_perRound)
        #self.Dices=[i for i in range(0,13)]
        for client in self.Dices:
            client.client_trainGPU(epoch=self.local_epoch)

        print("Finish train per client")
        self.updateClientModel()
        self.evaluate_CentralModel()
        #print([self.list_Client[idx].id for idx in self.Dices])
    def roundTrain(self):
        self.trainClientsQueue()
from torch.utils.data import Dataset,DataLoader


class plantdisease(Dataset):
    def __init__(self, train_df='data/raw/train.csv',test_df='data/raw/test.csv'
                 , img_dir='data/oneFolder'
                 , isTest=False,rank=0,central=False,data=None,num_client=20):
        """
        self.isTest (boolean) :test or not
        self.rank
        self.img_labels (csv) : data to generate -> 
        self.shardsList(list(csv)) : list of parts was splitted from train
        self.data (csv) : data server assign to each clients when generate (using self.makeClientDataSet() function to return data for clients )
            path: Server -> create plantdisease(isTest=True,central=True,num_client) -> plantdisease split train - test (use for server evaluation) -> {train} part continue splitted by the number of clients
            -> self.makeClientDataSet(rank) return part of rank^th shardsList
                  Client -> create local plantdiseas(central=False,data= return of server.dataset.makeClientDataSet(rank)) -> trainning
        """
        #Read dataset (balanced.csv is remake dataset that equalized all len(class))
        self.img_labels=pd.read_csv('/content/data1k.csv')
        self.img_dir='/content/train_images/'
        #Sort dataset by columns '1' (columns of labels)
        
        #self.img_labels=self.img_labels.drop([i for i in range(2101,5000)],axis=0)

        #If central -> split train - test data
        if central:
            self.train, self.test = train_test_split(self.img_labels, test_size=0.05)
            self.train.reset_index(inplace=True)
            self.test.reset_index(inplace=True)
            if isTest:
                self.img_labels=self.test
                self.data_csv=self.data_csv.sort_values('labels')
            else:
                self.img_labels=self.train
            self.img_labels=self.img_labels.sort_values('labels')
        #Split dataset by the number of num_client (so every client would have their own dataset) -> at most a client can contain 2 class of data
            self.shardsList=np.array_split(self.img_labels,num_client)
        else:
            self.data_csv=data
            self.data_csv.reset_index(inplace=True)
        #rank of client
        self.rank=rank
        self.isTest=isTest

        #Check if data exist -> also mean this is client
        if isinstance(data, pd.DataFrame):
            #Assign data to dataset
            self.img_labels=data



        #if rank:
        #    self.img_labels,old_csv=self.make_newCSV()

        #Muon train cung class cho train va test
        #self.train, self.test = train_test_split(self.img_labels, test_size=0.2)
        #self.train, self.test = train_test_split(self.img_labels, test_size=0.05)

            
        #self.test=self.test.append(old_csv)
    def makeClientDataSet(self,rank):
        newCSV=self.shardsList[rank]
        return newCSV

 #   def makeClientDataSet(self,rank):
 #       import random
 #       a=random.randint(0, 5)
 #       b=random.randint(5, 14)
 #       c=random.randint(5, 14)
 #       classList=[rank]
 #       old_csv=self.train
 #       newCSV=old_csv[(old_csv.iloc[:,2].isin(classList) ) ]

        
 #       return newCSV

 #Use to change state of dataset
    def changeisTest(self,boole):
        self.isTest=boole
        #Update img_labels
        if self.isTest:
            self.img_labels=self.test
        else:
            self.img_labels=self.train
    def __len__(self):
        return len(self.img_labels)
    #def make_newCSV(self):
    #    old_csv=self.img_labels
    #    newCSV=old_csv[old_csv.iloc[:,2]==(self.rank)]

        return newCSV,old_csv
    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            self.transform=transforms.Compose([
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.Resize([ 200, 300])])
            image = self.transform(image)
            label = self.img_labels['labels'][idx]

            return image, label

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
i=1
name='/content/drive/MyDrive/Prethesis-model/server{}.pickle'.format(i)
name

'/content/drive/MyDrive/Prethesis-model/server1.pickle'

# **RUN**

FROM BEGINNING

In [ ]:
info={'dataset': 'plantdisease',
 'client': {'learning_rate': 1e-05,
  'momentum': 0.0,
  'weight_decay': 1e-05,
  'batch_size': 25,
  'local_epoch': 1},
 'server': {'num_clients': 100,
  'num_class': 6,
  'num_round': 100,
  'num_client_perRound': 25,
  'isParallel': 'False',
  'model': 'mobilenetv3',
  'local_epoch': 10}}
from importlib.abc import Loader
import yaml
from yaml.loader import SafeLoader


if __name__ == '__main__':
    def saveServer(server):
        import pickle
        with open('/content/drive/MyDrive/Prethesis-model/server2.pickle', 'wb') as f:
            pickle.dump(server, f)

    from torch.utils.tensorboard import SummaryWriter
    #writer = SummaryWriter(
    a=server(tensorboard=0,parameters=info)
    a.generateClient()
    for i in range(0,info['server']['num_round']):
      print("Round {}".format(i))
      a.roundTrain()

      saveServer(a)
    #writer.flush()

In [6]:
!cp -b /content/drive/MyDrive/Prethesis-model/server1.pickle /content/drive/MyDrive/Prethesis-model/server1backup.pickle


Continue Running

In [ ]:
info={'dataset': 'plantdisease',
 'client': {'learning_rate': 1e-05,
  'momentum': 0.0,
  'weight_decay': 1e-05,
  'batch_size': 25,
  'local_epoch': 1},
 'server': {'num_clients': 100,
  'num_class': 6,
  'num_round': 100,
  'num_client_perRound': 25,
  'isParallel': 'False',
  'model': 'mobilenetv3',
  'local_epoch': 10}}
best_acc=[999]

from importlib.abc import Loader
import yaml
from yaml.loader import SafeLoader
#Continue 
if __name__ == '__main__':
  def saveServer(server):
    import pickle
    with open('/content/drive/MyDrive/Prethesis-model/server1.pickle', 'wb') as f:
      pickle.dump(server, f)
      f.close()
  def saveBestAServer(server,acc):
    import pickle
    with open('/content/drive/MyDrive/Prethesis-model/serverBest.pickle', 'wb') as f:
      pickle.dump(server, f)
      print('another best, {} accuracy'.format(acc))
      f.close()
  def saveBestLServer(server,loss):
    import pickle
    with open('/content/drive/MyDrive/Prethesis-model/serverBestL.pickle', 'wb') as f:
      pickle.dump(server, f)
      print('another best, {} loss'.format(loss))
      f.close()

  import pickle
  with open('/content/drive/MyDrive/Prethesis-model/serverBest.pickle', 'rb') as f:
    conti=pickle.load(f)
    #from torch.utils.tensorboard import SummaryWriter
    #writer = SummaryWriter()
    f.close()
  for i in range(0,info['server']['num_round']):
    print("Round {}".format(i))
    conti.roundTrain()
    if conti.accuracyHist[-1]>max(conti.accuracyHist[:-2]):
      saveBestServer(conti,conti.accuracyHist[-1])
    if conti.lossHist[-1]<min(conti.lossHist[:-2]):
      saveBestLServer(conti,conti.lossHist[-1])
    if i%4==0:
      saveServer(conti)

Round 0
Start to train [<__main__.Client object at 0x7fdbe451a9d0>, <__main__.Client object at 0x7fdbe52a5d90>, <__main__.Client object at 0x7fdbe4c5b610>, <__main__.Client object at 0x7fdc30f16790>, <__main__.Client object at 0x7fdc3c5c41d0>, <__main__.Client object at 0x7fdc3cae6a10>, <__main__.Client object at 0x7fdc313678d0>, <__main__.Client object at 0x7fdbe4de2850>, <__main__.Client object at 0x7fdc3c361910>, <__main__.Client object at 0x7fdbf9fb04d0>, <__main__.Client object at 0x7fdc3d13e290>, <__main__.Client object at 0x7fdbf9ef1810>, <__main__.Client object at 0x7fdbf95bbb50>, <__main__.Client object at 0x7fdbe5876390>, <__main__.Client object at 0x7fdc310fc590>, <__main__.Client object at 0x7fdc31e54e10>, <__main__.Client object at 0x7fdc3c6c2fd0>, <__main__.Client object at 0x7fdc3d1fdb50>, <__main__.Client object at 0x7fdbe458b910>, <__main__.Client object at 0x7fdc30f2b050>, <__main__.Client object at 0x7fdbf951d390>, <__main__.Client object at 0x7fdc31da7090>, <__main_

Temp

In [ ]:
!cp -b /content/drive/MyDrive/Prethesis-model/server1.pickle /content/drive/MyDrive/Prethesis-model/server1backup.pickle

In [ ]:
import pickle
with open('/content/drive/MyDrive/Prethesis-model/server2.pickle', 'rb') as f:
  contiServer=pickle.load(f)
for i in range(0,info['server']['num_round']):
  print("Round {}".format(i))
  a.roundTrain()
  saveServer(a)

In [ ]:
import os
scores=0
unpickler=0
if os.path.getsize('/content/drive/MyDrive/Prethesis-model/server2.pickle') > 0:      
    with open('/content/drive/MyDrive/Prethesis-model/server2.pickle', "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()
        print(unpickler,scores)

<_pickle.Unpickler object at 0x7f4df5d6b4d0> <__main__.server object at 0x7f4df624e550>


In [ ]:
unpickler

In [ ]:
import pickle
file = open("Fruits.obj",'rb')


In [ ]:
info={'dataset': 'plantdisease',
 'client': {'learning_rate': 1e-05,
  'momentum': 0.0,
  'weight_decay': 1e-05,
  'batch_size': 25,
  'local_epoch': 1},
 'server': {'num_clients': 100,
  'num_class': 6,
  'num_round': 100,
  'num_client_perRound': 25,
  'isParallel': 'False',
  'model': 'mobilenetv3',
  'local_epoch': 10}}
from importlib.abc import Loader
import yaml
from yaml.loader import SafeLoader



if __name__ == '__main__':
    def saveServer(server,i):
        import pickle
        name='/content/drive/MyDrive/Prethesis-model/server{}.pickle'.format(i)
        with open(name, 'wb') as f:
            pickle.dump(server, f)
        print('savedModel')
        
    from torch.utils.tensorboard import SummaryWriter
    #writer = SummaryWriter(
    a=server(tensorboard=0,parameters=info)
    a.generateClient()
    for i in range(0,info['server']['num_round']):
    
      print("Round {}".format(i))
      a.roundTrain()
      if i%4==0:
        saveServer(a,1)
    
    #writer.flush()